In [1]:
import torch, torchvision
from torch.autograd import Variable
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import Sampler
import itertools, math
import matplotlib.pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

## CNN Configuration

In [2]:
class TestDataSet(Dataset):
    def __init__(self):
        super().__init__()
        self.z = range(10)
    def __getitem__(self, i):
        return self.z[i]
    def __len__(self):
        return len(self.z)

In [3]:
data = TestDataSet()

## Dataset

In [4]:
indices = range(10)

In [5]:
class ShuffleCycleSubsetSampler(Sampler):
    """A cycle version of SubsetRandomSampler with
        reordering on restart"""
    def __init__(self, indices):
        self.indices = indices

    def __iter__(self):
        return self._gen()

    def _gen(self):
        i = len(self.indices)
        #perm = np.random.permutation(self.indices)
        while True:
            if i >= len(self.indices):
                perm = np.random.permutation(self.indices)
                i=0
            yield perm[i]
            i+=1
    
    def __len__(self):
        return len(self.indices)

In [6]:
sam2 = ShuffleCycleSubsetSampler(indices)

In [7]:
dl = DataLoader(data, batch_size=5, sampler = sam2)

In [ ]:
it = iter(dl); it2 = iter(sam2)

In [8]:
state = {'lab_sampler':dl.batch_sampler}

In [10]:
torch.save(state, '/home/maf388/bayesganfuture/models/testSave.ckpt')

In [11]:
state2 = torch.load('/home/maf388/bayesganfuture/models/testSave.ckpt')

In [12]:
state2['lab_sampler']

In [ ]:
G = torch.load('/home/maf388/bayesganfuture/models/dcganG.pt')

In [ ]:
class GanDataLoader(DataLoader):
    def __init__(self, generator, batch_size):
        self.G = generator.cuda()
        self.batch_size = batch_size

    def __iter__(self):
        return self
    
    def __next__(self):
        z = Variable(torch.randn(self.batch_size, self.G.z_dim).cuda())
        return self.G(z).detach()
    
    def __len__(self):
        inf = 2**63-1
        return inf

In [ ]:
thisLoader = GanDataLoader(G,32)

In [ ]:
batch = next(thisLoader)
imgs = torchvision.utils.make_grid(batch.cpu().data, normalize=True, nrow=8)
plt.imshow(np.transpose(imgs, (1, 2, 0)))

In [ ]:
D = torch.load('/home/maf388/bayesganfuture/models/dcganD.pt')

In [ ]:
import os
import sys

In [ ]:
sys.path